# Exercise 3.5 Stippling (cartopy)

Stippling in in pyplot is done with the function `contourf`.

Note that most of what we show here for georeferenced plots also applies for normal stippling.

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import cartopy.util as cutil

import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import xarray as xr

%matplotlib inline

In [ ]:
import utils

## Load data

### Function to create artificial data:

In [ ]:
# artificial data

def sample_data_3d(nlons, nlats):
    """Returns `lons`, `lats`, and fake `data`

    adapted from:
    http://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html
    """
    
    dlat = 180. / nlats / 2
    dlon = 360. / nlons

    lat = np.linspace(-90 + dlat, 90 - dlat, nlats)   
    lon = np.linspace(0, 360 - dlon, nlons)

    lons, lats = np.meshgrid(np.deg2rad(lon), np.deg2rad(lat))
    wave = 0.75 * (np.sin(2 * lats) ** 8) * np.cos(4 * lons)
    mean = 0.5 * np.cos(2 * lats) * ((np.sin(2 * lats)) ** 2 + 2)
    data = wave + mean
    
    return lon, lat, data

## CMIP 5, historical precipitation climatology (1986 to 2005)

Load historical, and projected climatlological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
fN = '../data/cmip5_delta_pr_rcp85_map.nc'

pr = xr.open_dataset(fN)

pr

## stippling $\rightarrow$ hatching

Stippling is called hatching in matplotlib.

`contourf`, takes a `hatches` keyword, and you have to specify one hatch-pattern per drawn level. The pattern are determined via characters, e.g. using `'/'` yields diagonal lines.

The hatching is more dense if the charachter is repeated, e.g.: `'///'`.

Specifying an empty string (`''`) ommits the hatching.

In [ ]:
# create sample data
lon, lat, data = sample_data_3d(90, 48)

# ====

f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.coastlines()

DATA, LON = cutil.add_cyclic_point(data, lon)

# ====

# add 3 levels
levels = [-1, -0.5, 0.5, 1]

# set hatching for each level
hatches = ['/', '', '////' ]

h = ax.contourf(LON, lat, DATA, levels=levels, hatches=hatches, cmap='RdBu')

# ====

ax.set_global()

# add colorbar 
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

## Hatch patterns

Most of the hatch patterns are quire intuitive:

    patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]

They are nicely visualized in this example from [stackoverflow](https://stackoverflow.com/a/14279608). Not only `contourf`, other functions, such as `bar` also take a hatch keyword*. 

(* However, `pcolormesh` does not, although the documentation says so.)

In [ ]:
f, ax = plt.subplots()

patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*", '/.' ]

for i, pattern in enumerate(patterns):
    ax.bar(i, 3, color='none', edgecolor='k', hatch=pattern)


Note that \ is an escape character. Therefore, `'\'` is not a valid string. You can either escape the escape character, using `'\\'` or add a space `'\ '`.

### Exercise
 * add hatches for precipitation changes larger than +- 20 % 

In [ ]:
# get data
lon, lat, pr_rel = pr.lon.values, pr.lat.values, pr.pr_rel.values

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# set levels
levels = np.arange(-20, 101, 20)

# add hatches

h = ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, extend='both')

ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pr_rel = pr.lon.values, pr.lat.values, pr.pr_rel.values

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# set levels
levels = np.arange(-20, 101, 20)

# add hatches
hatches =  ['\\\\', '', '', '//', '//', '//', '//', '//']

h = ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches, extend='both')

ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Better Solution

When you look closely, you can see that some of the hatch-lines are broken. It's actually better if you add a second contourf command, with only 3 levels, so that the entire area gets hatched in one go. You can then remove the contourf patches with `colors='none'`:

In [ ]:
# get data
lon, lat, pr_rel = pr.lon.values, pr.lat.values, pr.pr_rel.values

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# set levels
levels = np.arange(-20, 101, 20)
h = ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, extend='both')


# Second contourf command, only for the hatching. Note how there is no h = ..., so that
# the colorbar is correct. 

# set levels
levels = [-20, 20]

# add hatches
hatches =  ['\\\\', '', '//']
ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches,
                extend='both', colors='none')

ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

## Significance hatching

Often we don't want to hatch the same values that we color, but, e.g. stipple siginificant parts of the map. I recommend to do this in five steps:

* plot contourf of the significance values
* determine the levels
* add the hatches
* remove the colors with `colors='none'` & get rid of the colorbar
* add the actual data you want to plot

### Loading Data

For this example we use data contributed by a course participant. 

The data is ranked probability skill scores (RPSS) of ECMWF system 4 seasonal tercile forecasts verified against ERA-Interim on a global 1° x 1° grid. Additionally, the dataset contains the variable `signif` that indicates grid points with a significant improvement in skill. `signinf` is given as boolean, 0 means, no imporvement, 1 means an improvement.

Due to the resolution the plotting will take a moment.

In [ ]:
fN = '../data/globalRPSS.nc'

ds = xr.open_dataset(fN)
ds

In [ ]:
# create sample data

lat = ds.latitude.values

SIG, LON = cutil.add_cyclic_point(ds.signif.values, ds.longitude.values)

# convinience function to add colorbar
def add_colorbar(h, ax, d=0):
    cbax = f.add_axes([0, 0, 0.1, 0.1 + d])
    cbar = plt.colorbar(h, cax=cbax)
    f.canvas.mpl_connect('draw_event', utils.resize_colorbar_vert(cbax, ax, size=0.02, pad=0.01))

    return cbar
# ========================================

f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()), gridspec_kw=dict(wspace=0.4))

f.set_size_inches(20 / 2.54, 12 / 2.54)
axes = axes.flatten()
# =====

ax = axes[0]
h = ax.contourf(LON, lat, SIG, 4)
add_colorbar(h, ax, d=0)
ax.set_title('Step 1: raw contourf')

# =====

ax = axes[1]
levels = [0, 0.5, 1]
h = ax.contourf(LON, lat, SIG, levels=levels)
add_colorbar(h, ax, d=0.1)
ax.set_title('Step 2: levels')

# =====

ax = axes[2]
levels = [0, 0.5, 1]
hatches = ['', '...']
h = ax.contourf(LON, lat, SIG, levels=levels, hatches=hatches)
add_colorbar(h, ax, d=0.2)
ax.set_title('Step 3: hatching')

# =====
ax = axes[3]
levels = [0, 0.5, 1]
hatches = ['', '...']
#h = ax.contourf(LON, lat, SIG, levels=levels)
h = ax.contourf(LON, lat, SIG, levels=levels, hatches=hatches, colors='none')

ax.set_title('Step 4: remove color & bar')

# =====

for ax in axes:
    ax.set_global()
    ax.coastlines()

I would not actually do a four-panel plot. But repeat the same plot over and over, doing one step after the other.

### Final figure

In [ ]:
# get data
lat = ds.latitude.values

SIG, LON = cutil.add_cyclic_point(ds.signif.values, ds.longitude.values)

# ========================================

f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()), gridspec_kw=dict(wspace=0.4))

# =====

# plot RPSS data

levels = np.arange(-1, 1.1, 0.25)

# create norm to plot levels
pal = sns.color_palette('RdBu_r', n_colors=len(levels) - 1)
new_cmap, cnorm = mpl.colors.from_levels_and_colors(levels, pal)

h = ax.pcolormesh(*utils.infer_interval_breaks(ds.longitude, ds.latitude, clip=True), ds.RPSS,
                  transform=ccrs.PlateCarree(), cmap=new_cmap, norm=cnorm, vmax=1, vmin=-1)

cbar = add_colorbar(h, ax, d=0.4)
#cbar.set_ticks([-1, -0.5, 0, 0.5, 1])

levels = [0, 0.5, 1]
hatches = ['', '...']
ax.contourf(LON, lat, SIG, levels=levels, hatches=hatches, colors='none')


ax.set_title('Step 5: plot the actual data')

# =====

ax.set_global()
ax.coastlines();

### Exercise

The precipitation data from CMIP5 contains p values, indicating where the change between the historical and future period is significant (note: this derived from a simple t-test, which is not necessarily appropriate for the data).

* use contourf to plot the p-values

Note: it's p-values and not a boolean indicating significance as above.

In [ ]:
# get data
lon, lat, pval = pr.lon.values, pr.lat.values, pr.pval.values


# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PVAL, LON = cutil.add_cyclic_point(pval, lon)

# plot pvalues here

            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval = pr.lon.values, pr.lat.values, pr.pval.values


# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PVAL, LON = cutil.add_cyclic_point(pval, lon)

# plot pvalues here
h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree())
            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Exercise

* We want to stipple everything with a p-value smaller than 0.1: determine the levels
* add the hatches

In [ ]:
# get data
lon, lat, pval = pr.lon.values, pr.lat.values, pr.pval.values


# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PVAL, LON = cutil.add_cyclic_point(pval, lon)

# set the levels

# add hatches

h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree())
            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval = pr.lon.values, pr.lat.values, pr.pval.values


# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

PVAL, LON = cutil.add_cyclic_point(pval, lon)

# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ['..', '']

h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches)
            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Exercise

* remove the colors with `colors='none'`
* plot the relative precipitation change below the hatches
* make sure the right data is used for the colorbar

In [ ]:
# get data
lon, lat, pval, pr_rel = pr.lon.values, pr.lat.values, pr.pval.values, pr.pr_rel.values

PVAL, LON = cutil.add_cyclic_point(pval, lon)
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# ================

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

# plot relative precipitation change


# plot significance stippling
# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ['...', '']
h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches)
            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval, pr_rel = pr.lon.values, pr.lat.values, pr.pval.values, pr.pr_rel.values

PVAL, LON = cutil.add_cyclic_point(pval, lon)
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# ================

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

# plot relative precipitation change
# set levels
levels = np.arange(-20, 101, 20)
h = ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, extend='both')

# plot significance stippling
# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ['...', '']
ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches, colors='none')
            
ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()

## Manual hatches

You can also manually add hatches, at all points that are significant. However, this leads to a hatching pattern that depends on the resolution of the data which is not desirable.

In [ ]:
# get data
lon, lat, pval, pr_rel = pr.lon.values, pr.lat.values, pr.pval.values, pr.pr_rel.values

PVAL, LON = cutil.add_cyclic_point(pval, lon)
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# ================

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))

ax.coastlines()

# plot relative precipitation change
# set levels
levels = np.arange(-20, 101, 20)
h = ax.contourf(LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, extend='both')

# 'manual' stippling

# create array with all lat/ lon combinations
LONs, lats = np.meshgrid(LON, lat)

# find significant points
sig = PVAL.flatten() <= 0.1

LONs = LONs.flatten()[sig == 1]
lats = lats.flatten()[sig == 1]

# add scatterpoints
ax.plot(LONs, lats, '.', color='0.1', transform=ccrs.PlateCarree(), ms=1)


ax.set_global()

# add colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, cax=cbax, spacing='proportional')

func = utils.resize_colorbar_vert(cbar.ax, ax, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
plt.draw()